<a href="https://colab.research.google.com/github/salim928/GEN-AI/blob/adamstemp/Text_summarization_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Mon Feb 24 14:15:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!cd "/Text_summarization_2"

/bin/bash: line 1: cd: /Text_summarization_2: No such file or directory


In [5]:
!pwd

/content


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00


In [7]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize


import pandas as pd
from tqdm import tqdm
import numpy as np
import torch

In [8]:
#torch.cuda.empty_cache()
#torch.cuda.reset_max_memory_allocated()

In [9]:
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [11]:
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()

/usr/local/lib/python3.11/dist-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [12]:
#Access the tokenizer

model_ckpt = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [13]:
# Access the model
model_falcon = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
#loading the data
data_yale = load_dataset("Yale-LILY/aeslc")

README.md:   0%|          | 0.00/6.34k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.26M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/896k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/794k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14436 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1960 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1906 [00:00<?, ? examples/s]

In [15]:
data_yale

DatasetDict({
    train: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 14436
    })
    validation: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1960
    })
    test: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1906
    })
})

In [16]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [17]:
data_yale['train'][0]

{'email_body': 'Greg/Phillip,  Attached is the Grande Communications Service Agreement.\nThe business points can be found in Exhibit C.  I Can get the Non-Disturbance agreement after it has been executed by you and Grande.\nI will fill in the Legal description of the property one I have received it.\nPlease execute and send to:  Grande Communications, 401 Carlson Circle, San Marcos Texas, 78666 Attention Hunter Williams.\n<<Bishopscontract.doc>>\n',
 'subject_line': 'Service Agreement'}

In [18]:
data_yale['train']['email_body'][1]

'Phillip & Keith  Attached is the first draw request, I will need some of these funds immediately.\nI think checks out of Bishops Corner, L.P. may be the easiest, or you can wire money to me and I can write the checks when needed.\nPlease let me know how you wish to handle this and I will proceed.\nAdditionally, it is getting close to closing.\nI will need to get contracts signed at this point as soon as possible.\nWe can handle this in two ways.\nI prefer that you elect me as a Vice - President of the General Partner and then I will sign all documents, or I can Federal Express contracts (this is more difficult).\nI have a funeral at 10:00 and I will be back about 1:00.\nI look forward to hearing from you.\nGreg     - Draw # 1 - 10.25.01.xls\n'

In [19]:
data_yale['train']['subject_line'][1]

'Bishops Corner'

In [20]:
splits_lengths = [len(data_yale[split]) for split in data_yale]

print(f"Splits Lenght:, {splits_lengths}")

Splits Lenght:, [14436, 1960, 1906]


In [21]:
print("Features:", data_yale['train'].column_names)

Features: ['email_body', 'subject_line']


In [22]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['email_body'], max_length=512, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['subject_line'], max_length=128, truncation=True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [23]:
data_yale_pt = data_yale.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14436 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1960 [00:00<?, ? examples/s]

Map:   0%|          | 0/1906 [00:00<?, ? examples/s]

In [24]:
data_yale_pt['train']['input_ids'][0]

[11859,
 87,
 23305,
 7446,
 6,
 28416,
 15,
 26,
 19,
 8,
 15940,
 11538,
 1387,
 7139,
 5,
 37,
 268,
 979,
 54,
 36,
 435,
 16,
 1881,
 13506,
 17,
 205,
 5,
 27,
 1072,
 129,
 8,
 5388,
 18,
 308,
 343,
 19413,
 565,
 2791,
 227,
 34,
 65,
 118,
 13763,
 57,
 25,
 11,
 15940,
 5,
 27,
 56,
 14,
 16,
 8,
 11281,
 4210,
 13,
 8,
 785,
 80,
 27,
 43,
 1204,
 34,
 5,
 863,
 12133,
 11,
 1299,
 12,
 10,
 15940,
 11538,
 6,
 3,
 20016,
 7291,
 739,
 15697,
 6,
 1051,
 16902,
 7,
 2514,
 6,
 489,
 3840,
 3539,
 20748,
 14046,
 6060,
 5,
 3,
 2,
 279,
 23,
 6921,
 7,
 20871,
 5,
 7171,
 3155,
 3155,
 1]

In [25]:
data_yale_pt['train']['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [26]:
data_yale_pt['train']['labels'][0]

[1387, 7139, 1]

In [27]:
# For more data

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_falcon)

In [28]:
# Training Arguments

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='yale_summary',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [29]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [30]:
# trian the model

trainer = Trainer(
    model=model_falcon,
    args=training_args,
    data_collator=data_collator,
    train_dataset=data_yale_pt['test'],
    eval_dataset=data_yale_pt['validation'])


In [31]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [32]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()

/usr/local/lib/python3.11/dist-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [33]:
# Train
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: salimadams49 (therap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=119, training_loss=5.989372702205882, metrics={'train_runtime': 142.9759, 'train_samples_per_second': 13.331, 'train_steps_per_second': 0.832, 'total_flos': 76003673407488.0, 'train_loss': 5.989372702205882, 'epoch': 0.9989506820566632})

In [35]:
from tqdm import tqdm

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device="cuda",
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                   length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Fix empty summary issue
        decoded_summaries = [" " if d == "" else d for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute metric after processing all batches
    score = metric.compute()
    return score


In [37]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [39]:
!pip install rouge_score # Install the missing 'rouge_score' package

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a319104c4054a607bda8ce398467654f5a66545dee2d2c008844ed6d389cbbae
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [40]:
from evaluate import load

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load('rouge') # use load to get the rouge metric

In [41]:
score = calculate_metric_on_test_ds(
    data_yale_pt['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size = 2,
    column_text = 'email_body',
    column_summary= 'subject_line'
)
rouge_dict = dict((rn, score[rn] * 100) for rn in rouge_names)  # Access the score directly instead of using .mid.fmeasure
# Access the score directly instead of using .mid.fmeasure
# You can multiply by 100 to express scores as percentages

pd.DataFrame(rouge_dict, index = [f'falcon'] )

100%|██████████| 5/5 [00:10<00:00,  2.18s/it]


,rouge1,rouge2,rougeL,rougeLsum
falcon,6.299879,1.722165,5.196591,5.582958


In [42]:
# save the trained model
model_falcon.save_pretrained('yale_summary')

In [43]:
# save the tokenizer
tokenizer.save_pretrained('yale_summary')

('yale_summary/tokenizer_config.json',
 'yale_summary/special_tokens_map.json',
 'yale_summary/spiece.model',
 'yale_summary/added_tokens.json',
 'yale_summary/tokenizer.json')

In [44]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained('yale_summary')

In [46]:
# prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = data_yale["test"][0]["email_body"]

reference = data_yale["test"][0]["subject_line"]

pipe = pipeline("summarization", model="Falconsai/text_summarization", tokenizer=tokenizer)


print("email_body:")
print(sample_text)


print("\nReference subject_line:")
print(reference)


print("\nModel Subject_line:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


email_body:
Phillip,   Could you please do me a favor?
I would like  to read your current title policy to see what it says about easements.
You  should have received a copy during your closing.
I don't know how many  pages it will be but let me know how you want to handle getting a copy  made.
I'll be happy to make the copy, or whatever makes it easy for  you.
Thanks,


Reference subject_line:
Huntley/question


Model Subject_line:
I would like to read your current title policy to see what it says about easements . You should have received a copy during your closing . I'll be happy to make the copy, or whatever makes it easy for you .
